In [1]:
import os
import requests
import numpy as np
import pandas as pd


from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

# 01 Prepare data

- NHK program list text and genres (too few)
- MARC-ja (not available)
- JCoLA https://github.com/osekilab/JCoLA <-- we are using this

In [2]:
# def get_nhk_programs(url: str) -> dict:
#     """
#     NHK番組表を取得する
#     """

#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         data = response.json()
#     except:
#         data = None
#     return data


# def cast_json_to_pandas(json_data):
#     """
#     JSONデータをPandasのDataFrameに変換する
#     """
#     if json_data is None:
#         return None
#     return pd.json_normalize(json_data["list"]["g1"])

In [3]:
# # https://api.nhk.or.jp/v2/pg/genre/{area}/{service}/{genre}/{date}.json?key={apikey}

# apikey = os.environ.get("NHK_API_KEY", None)
# area = "130"  # 東京
# service = "g1"  # NHK総合
# start_date = "2024-04-24"  # 開始日
# end_date = "2024-11-24"  # 終了日

# get_list_of_dates = (
#     lambda start_date, end_date: pd.date_range(start_date, end_date)
#     .strftime("%Y-%m-%d")
#     .tolist()
# )

# dates = get_list_of_dates(start_date, end_date)

# nhk_programs_json = map(
#     lambda date: get_nhk_programs(
#         f"https://api.nhk.or.jp/v2/pg/list/{area}/{service}/{date}.json?key={apikey}"
#     ),
#     dates,
# )
# nhk_programs_json = filter(lambda x: x is not None, nhk_programs_json)
# df = pd.concat(map(cast_json_to_pandas, nhk_programs_json))

In [4]:
# data_path = "../local/data/JCoLA-main/data/jcola-v1.0/in_domain_train-v1.0.tsv"
# df = pd.read_csv(data_path, sep="\t")


# 02 Featrue engineering

In [10]:
# os.makedirs("../local/data", exist_ok=True)
# df.to_csv("../local/data/nhk_programs.csv", index=False)
# df = pd.read_csv("../local/data/nhk_programs.csv", dtype=str).dropna()

data_path = "../local/data/JCoLA-main/data/jcola-v1.0/in_domain_train-v1.0.tsv"
df = pd.read_csv(data_path, sep="\t")

# Text feature extraction
model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite"
# model_name = "studio-ousia/luke-japanese-base-lite"
encoder = HuggingFaceEmbeddings(model_name=model_name)

# text_features = ["title", "subtitle", "content", "act"]
text_features = ["sentence"]

for i in text_features:
    embeddings = pd.DataFrame(
        encoder.embed_documents(df[i]), index=df.index
    ).add_prefix(f"embed_{i}_")
    df = df.join(embeddings, how="left")


In [11]:
# Choose training features
# training_features = [i for i in df.columns if "embed_" in i]
# df = df[training_features + ["label"]]

# Target label data clearning
# df["genres"] = df["genres"].map(eval)
# df = df.explode("genres").reset_index(drop=True)

# Choose training features
training_features = [i for i in df.columns if "embed_" in i]
target_label = "label"
df = df[training_features + [target_label]]
df.head(1)

,embed_sentence_0,embed_sentence_1,embed_sentence_2,embed_sentence_3,embed_sentence_4,embed_sentence_5,embed_sentence_6,embed_sentence_7,embed_sentence_8,embed_sentence_9,...,embed_sentence_759,embed_sentence_760,embed_sentence_761,embed_sentence_762,embed_sentence_763,embed_sentence_764,embed_sentence_765,embed_sentence_766,embed_sentence_767,label
0,0.779722,-0.159863,0.390812,0.163809,0.441645,-0.422382,0.294479,0.004554,-0.151662,0.318913,...,-0.282139,-0.167588,0.326852,-0.147425,0.16059,0.037417,-0.125623,0.472888,-0.110388,1


In [12]:


# train classifier by logitic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df[training_features]
y = df[target_label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("train score ", clf.score(X_train, y_train))
print("test score ", clf.score(X_test, y_test))
print(classification_report(y_test, y_pred))


train score  0.8643179765130985
test score  0.8229768786127167
              precision    recall  f1-score   support

           0       0.45      0.18      0.26       236
           1       0.85      0.96      0.90      1148

    accuracy                           0.82      1384
   macro avg       0.65      0.57      0.58      1384
weighted avg       0.78      0.82      0.79      1384



In [13]:
# train text classifier with light boost
import lightgbm as lgb
from lightgbm import LGBMClassifier

clf = LGBMClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("train score ", clf.score(X_train, y_train))
print("test score ", clf.score(X_test, y_test))

print(classification_report(y_test, y_pred))

 


[LightGBM] [Info] Number of positive: 4621, number of negative: 914
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056797 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 5535, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.834869 -> initscore=1.620536
[LightGBM] [Info] Start training from score 1.620536
train score  0.9998193315266486
test score  0.8236994219653179
              precision    recall  f1-score   support

           0       0.37      0.05      0.08       236
           1       0.83      0.98      0.90      1148

    accuracy                           0.82      1384
   macro avg       0.60      0.52      0.49      1384
weighted avg       0.75      0.82      0.76      1384



In [9]:
# Using HuggingFace api for feature extraction

# model_id = "oshizo/sbert-jsnli-luke-japanese-base-lite"
# hf_token = os.environ.get("HUGGINGFACE_API_KEY_READ", None)
# api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
# headers = {"Authorization": f"Bearer {hf_token}"}

# def encode_texts(texts):
#     """
#     テキストをエンコードする
#     """
#     response = requests.post(
#         api_url,
#         headers=headers,
#         json={"inputs": texts, "options": {"wait_for_model": True}},
#     )
#     return response.json()

# tmp = pd.read_csv("../local/data/nhk_programs.csv", dtype=str)
# tmp = tmp.dropna()['title'].tolist()
# tmp = encode_texts(tmp)